In [2]:
%cd ..

/Users/john.onwuemeka/Downloads/Mini_DIVA


/Users/john.onwuemeka/Library/Python/3.12/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

import copy
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

from utils import *

from imputers.customMICEImputer import *

In [4]:
datapath = "./datasets/"

data, X, y, Xinds, yind = read_dataset(dataset="iris", datapath=datapath)

In [5]:
numerical_cols, categorical_cols = get_num_cat_vars(X)

In [6]:
from sklearn.preprocessing import LabelEncoder


def encode_categorical_columns(df, categorical_cols):
    df_encoded = df.copy()
    encoding_mappings = {}

    for col in categorical_cols:
        encoder = LabelEncoder()
        df_encoded[col] = encoder.fit_transform(df_encoded[col])
        encoding_mappings[col] = {
            i: category for i, category in enumerate(encoder.classes_)
        }

    return df_encoded, encoding_mappings

In [7]:
def numerical_backto_categorical(df, encoding_mappings):
    for col_index, col_name in enumerate(df.columns):
        if col_name in encoding_mappings:
            mapping = encoding_mappings[col_name]
            df[col_name] = df[col_name].apply(lambda x: mapping.get(int(x), x))
    return df

In [9]:
dataOriginal = copy.deepcopy(data)


df_encoded, encoding_mappings = encode_categorical_columns(X, categorical_cols)
dfTrain, dfTest = df_encoded.iloc[: int(0.8 * len(X))], df_encoded[int(0.8 * len(X)) :]

In [10]:
fraction = 0.2  # 20% sparsity
mice = customMICEImputer(
    numerical_cols,
    categorical_cols,
    iteration=2,
    methodN="ridge",
    methodC="random_forest",
    solver="newton-cg",
)

df_nanTrain, nan_indsTrain = set_fraction_missing(dfTrain, fraction=fraction)
outputTrain = mice.fit(df_nanTrain)
outputTrain.set_index(
    dfTrain.index, inplace=True
)  # Need to set the index as the dfTrain
outputTrain = numerical_backto_categorical(outputTrain, encoding_mappings)


df_nanTest, nan_indsTest = set_fraction_missing(dfTest, fraction=fraction)
output = mice.transform(df_nanTest)
output.set_index(dfTest.index, inplace=True)  # Need to set the index as the dfTest
output = numerical_backto_categorical(output, encoding_mappings)